In [36]:
import pandas as pd
import numpy as np
import json

In [37]:
# Pick up from our scraped data
data = json.load(open('scraped_players.json'))

In [38]:
# Here's an example row. Look at the stats property... it's a nested dictionary
data[0]

{'#': '1',
 'Academic Year': 'Jr.',
 'Full Name': 'Brandon Riley',
 'Hometown / High School': 'Burlington, N.C. / Williams',
 'Ht.': '6-0',
 'Pos.': 'OF',
 'Wt.': '175',
 'bio': '\r\n                        Biography\r\n                                                    \r\n                            Dynamic outfielder who figures to be among the team leaders as a junior.\r\n\r\nSophomore Season (2017)\r\nHit .317 with seven homers, nine doubles and a team-high 52 RBIs •\xa0Also scored 46 runs, drew 37 walks and led the team with five triples, which is tied for 10th-most in UNC history for a single season • Appeared in all 63 games and started 62, primarily in right field • Stole 10 bases in 14 attempts • Had 21 multi-hit games, including a season-high three hits on eight occasions • Also drove in multiple runs 14 times, including a season-best four at ECU on March 22 • Followed up the ECU game with a big series against Miami, recording six hits and driving in four runs with a homer 

In [39]:
# We need to recreate tables from our data
# We will create dataframes for each permutation of the data
# So we currently have nested data like this:
#
# player (name, position, etc):
#    -> stats
#        -> career_stats
#            -> stats type (hitting / pitching)
#                -> stats for year 1
#                -> stats for year 2
#                -> ...

# We are going to denormalize "grouped" relationship by unrolling them and repeating each level of data
# So our example above becomes
# name, hitting, year 1, stats...
# name, hitting, year 2, stats...
# name, pitching, year 1, stats...
# name, pitching, year 2, stats...
# etc

In [40]:
# We're going to create a list of stats
data_for_df = []

# We loop over each player in the data we loaded
for player in data:
    # Now we loop over each value in the career_stats items.
    # Not all players have stats so we use `.get` to "defensively" try to get the items
    # So we're saying:
    # give me the stats if it exists, otherwise an empty dictionary
    # Then using that result give me the career_stats key or and empty dictionary
    # and finally the items for the dictionary
    # Note that we can only chain .get and .items because those are attributes on dictionaries
    # and we're defaulting to dictionaries if the item is not found
    for key, val in player.get('stats', {}).get('career_stats', {}).items():
        # Stat type is going from 'Hitting Statistics'
        # to 'hitting statistics'
        # to ['hitting', 'statistics']
        # to 'hitting'
        stat_type = key.lower().split()[0]
        # Players have to have a name and number
        name = player.get('Full Name')
        if not name:
            continue
        num = player.get('#')
        if not num:
            continue
        # Now loop through each value in the career_stats item
        for stat in val:
            # Add the name
            stat['name'] = name
            # Add the player number
            stat['num'] = num
            # Add the stat_type
            stat['stat_type'] = stat_type
            # Append the denormalized row
            # This is now something like
            # {'name': 'John Doe', 'num': '1', 'stat_type': 'hitting', 'year': '2016', ...}
            data_for_df.append(stat)

In [41]:
# We feed our list of dictionaries into the dataframe class to instantiate a new dataframe
df = pd.DataFrame(data_for_df)

In [42]:
# Attempt to convert everything to a numeric value and by specifying
# errors='ignore' the original value will remain in place if it's not numeric
# Note that this returns a new dataframe so we have to assign it back to our df variable
df = df.apply(pd.to_numeric, errors='ignore')

In [43]:
# Drop the avg stat since it's not always present
# Note this time we can use inplace=True and it does NOT return a new dataframe
df.drop('avg', axis=1, inplace=True)

In [44]:
# Generate the batting average and era for all records
# Note again how we have single arithmetic operators
# but our data is a list- this, again, is the power and short cut of pandas / numpy
df['ba'] = df.h / df.ab
df['era2'] = df.r / df.ip

In [45]:
# Now We can generate groups
# We can make a compound group by name and stat type
by_player = df.groupby(('name', 'stat_type'))
# and by year
by_year = df.groupby('year')
# and by type
by_type = df.groupby('stat_type')

In [46]:
# We can also group by a property after fetching a group
# Here's the batting average for the whole team by year
by_type.get_group('hitting').groupby('year').mean().ba

year
2015    0.215686
2016    0.219881
2017    0.241031
Name: ba, dtype: float64

In [47]:
# Here are all the batting averages across the player / type compound group
by_player.mean().h / by_player.mean().ab

name                  stat_type
Ashton McGee          hitting      0.327354
Austin Bergner        hitting           NaN
                      pitching          NaN
Bo Weiss              hitting           NaN
                      pitching          NaN
Brandon Martorano     hitting      0.068182
Brandon Riley         hitting      0.306220
Brendan Illies        hitting      0.210526
Brett Daniels         hitting           NaN
                      pitching          NaN
Cody Roberts          hitting      0.262295
                      pitching          NaN
Gianluca Dalatri      hitting           NaN
                      pitching          NaN
Hansen Butler         hitting           NaN
                      pitching          NaN
Ike Freeman           hitting      0.173913
Jason Morgan          hitting           NaN
                      pitching          NaN
Josh Hiatt            hitting           NaN
                      pitching          NaN
Josh Ladowski         hitting      0.269231


In [48]:
# We can mask out the rows where the era2 and ba is null
# and use the .loc method to specify a subset of columns to view
df[df.era2.isnull() & df.ba.isnull()].loc[:, ('name', 'year', 'era', 'ba')]

,name,year,era,ba
11,Cody Roberts,2017,0.0,NaN
20,Brett Daniels,2015,NaN,NaN
21,Brett Daniels,2016,NaN,NaN
22,Brett Daniels,2017,NaN,NaN
24,Tyler Baum,2017,NaN,NaN
27,Hansen Butler,2015,NaN,NaN
28,Hansen Butler,2016,NaN,NaN
31,Taylor Sugg,2016,NaN,NaN
32,Taylor Sugg,2017,NaN,NaN
36,Jason Morgan,2015,NaN,NaN


In [49]:
# We can compare that with all the batting averages
df.loc[:, ('name', 'year', 'ba')]

,name,year,ba
0,Brandon Riley,2016,0.293814
1,Brandon Riley,2017,0.316964
2,Kyle Datres,2016,0.250000
3,Kyle Datres,2017,0.265217
4,Brandon Martorano,2017,0.068182
5,Ashton McGee,2017,0.327354
6,Ike Freeman,2017,0.173913
7,Zack Gahagan,2015,0.215686
8,Zack Gahagan,2016,0.297297
9,Zack Gahagan,2017,0.242857


In [50]:
# Or all the eras
df.loc[:, ('name', 'year', 'era2')]

,name,year,era2
0,Brandon Riley,2016,NaN
1,Brandon Riley,2017,NaN
2,Kyle Datres,2016,NaN
3,Kyle Datres,2017,NaN
4,Brandon Martorano,2017,NaN
5,Ashton McGee,2017,NaN
6,Ike Freeman,2017,NaN
7,Zack Gahagan,2015,NaN
8,Zack Gahagan,2016,NaN
9,Zack Gahagan,2017,NaN


In [51]:
df.head(43)

,2b,3b,a,ab,app,bavg,bb,bf,bk,cg,...,sho,slg%,so,stat_type,sv,w,wp,year,ba,era2
0,16,3,0.0,194.0,NaN,NaN,20,NaN,NaN,NaN,...,NaN,0.423,28,hitting,NaN,NaN,NaN,2016,0.293814,NaN
1,9,5,1.0,224.0,NaN,NaN,37,NaN,NaN,NaN,...,NaN,0.496,34,hitting,NaN,NaN,NaN,2017,0.316964,NaN
2,6,2,64.0,152.0,NaN,NaN,27,NaN,NaN,NaN,...,NaN,0.375,28,hitting,NaN,NaN,NaN,2016,0.250000,NaN
3,13,1,109.0,230.0,NaN,NaN,29,NaN,NaN,NaN,...,NaN,0.422,45,hitting,NaN,NaN,NaN,2017,0.265217,NaN
4,0,0,12.0,44.0,NaN,NaN,6,NaN,NaN,NaN,...,NaN,0.068,13,hitting,NaN,NaN,NaN,2017,0.068182,NaN
5,10,2,12.0,223.0,NaN,NaN,36,NaN,NaN,NaN,...,NaN,0.484,57,hitting,NaN,NaN,NaN,2017,0.327354,NaN
6,3,0,50.0,46.0,NaN,NaN,7,NaN,NaN,NaN,...,NaN,0.304,10,hitting,NaN,NaN,NaN,2017,0.173913,NaN
7,4,1,29.0,153.0,NaN,NaN,14,NaN,NaN,NaN,...,NaN,0.294,32,hitting,NaN,NaN,NaN,2015,0.215686,NaN
8,6,0,40.0,148.0,NaN,NaN,22,NaN,NaN,NaN,...,NaN,0.439,29,hitting,NaN,NaN,NaN,2016,0.297297,NaN
9,9,0,105.0,210.0,NaN,NaN,33,NaN,NaN,NaN,...,NaN,0.386,51,hitting,NaN,NaN,NaN,2017,0.242857,NaN


In [16]:
display.max_rows

AttributeError: 'function' object has no attribute 'max_rows'

In [52]:
cols = df.columns.tolist()

In [53]:
df["name"]

0            Brandon Riley
1            Brandon Riley
2              Kyle Datres
3              Kyle Datres
4        Brandon Martorano
5             Ashton McGee
6              Ike Freeman
7             Zack Gahagan
8             Zack Gahagan
9             Zack Gahagan
10            Cody Roberts
11            Cody Roberts
12            Cody Roberts
13            Cody Roberts
14           Josh Ladowski
15           Josh Ladowski
16           Michael Busch
17           Brett Daniels
18           Brett Daniels
19           Brett Daniels
20           Brett Daniels
21           Brett Daniels
22           Brett Daniels
23              Tyler Baum
24              Tyler Baum
25           Hansen Butler
26           Hansen Butler
27           Hansen Butler
28           Hansen Butler
29             Taylor Sugg
30             Taylor Sugg
31             Taylor Sugg
32             Taylor Sugg
33            Jason Morgan
34            Jason Morgan
35            Jason Morgan
36            Jason Morgan
3

In [54]:
cols

['2b',
 '3b',
 'a',
 'ab',
 'app',
 'bavg',
 'bb',
 'bf',
 'bk',
 'cg',
 'e',
 'er',
 'era',
 'fld%',
 'gdp',
 'gp',
 'gs',
 'h',
 'hbp',
 'hr',
 'ip',
 'l',
 'name',
 'num',
 'ob%',
 'po',
 'r',
 'rbi',
 'sb',
 'sf',
 'sfa',
 'sh',
 'sha',
 'sho',
 'slg%',
 'so',
 'stat_type',
 'sv',
 'w',
 'wp',
 'year',
 'ba',
 'era2']

In [55]:
indices = [i for i, s in enumerate(cols) if 'name' in s]

In [56]:
indices

[22]

In [57]:
cols = cols[22:] + cols[:22]
## Moving name to the top of the list, so that it will display in the table without expanding

In [58]:
cols

['name',
 'num',
 'ob%',
 'po',
 'r',
 'rbi',
 'sb',
 'sf',
 'sfa',
 'sh',
 'sha',
 'sho',
 'slg%',
 'so',
 'stat_type',
 'sv',
 'w',
 'wp',
 'year',
 'ba',
 'era2',
 '2b',
 '3b',
 'a',
 'ab',
 'app',
 'bavg',
 'bb',
 'bf',
 'bk',
 'cg',
 'e',
 'er',
 'era',
 'fld%',
 'gdp',
 'gp',
 'gs',
 'h',
 'hbp',
 'hr',
 'ip',
 'l']

In [59]:
cols

['name',
 'num',
 'ob%',
 'po',
 'r',
 'rbi',
 'sb',
 'sf',
 'sfa',
 'sh',
 'sha',
 'sho',
 'slg%',
 'so',
 'stat_type',
 'sv',
 'w',
 'wp',
 'year',
 'ba',
 'era2',
 '2b',
 '3b',
 'a',
 'ab',
 'app',
 'bavg',
 'bb',
 'bf',
 'bk',
 'cg',
 'e',
 'er',
 'era',
 'fld%',
 'gdp',
 'gp',
 'gs',
 'h',
 'hbp',
 'hr',
 'ip',
 'l']

In [60]:
 indic = [i for i, s in enumerate(cols) if 'year' in s]

In [61]:
indic

[18]

In [62]:
cols.insert(1, cols.pop(18))
## Moving year so that it will come directly after the player's name, indicating which year the stats are coming from and providing context to repeated names

In [63]:
cols

['name',
 'year',
 'num',
 'ob%',
 'po',
 'r',
 'rbi',
 'sb',
 'sf',
 'sfa',
 'sh',
 'sha',
 'sho',
 'slg%',
 'so',
 'stat_type',
 'sv',
 'w',
 'wp',
 'ba',
 'era2',
 '2b',
 '3b',
 'a',
 'ab',
 'app',
 'bavg',
 'bb',
 'bf',
 'bk',
 'cg',
 'e',
 'er',
 'era',
 'fld%',
 'gdp',
 'gp',
 'gs',
 'h',
 'hbp',
 'hr',
 'ip',
 'l']

In [64]:
## Integrating reordered columns back into the dataframe
df = df[cols]

In [65]:
## Checking my work
df

,name,year,num,ob%,po,r,rbi,sb,sf,sfa,...,era,fld%,gdp,gp,gs,h,hbp,hr,ip,l
0,Brandon Riley,2016,1,0.355,13.0,27,35.0,3.0,5.0,NaN,...,NaN,0.929,3.0,54.0,51,57,1,1,NaN,NaN
1,Brandon Riley,2017,1,0.406,106.0,46,52.0,10.0,5.0,NaN,...,NaN,0.964,6.0,63.0,62,71,0,7,NaN,NaN
2,Kyle Datres,2016,3,0.384,34.0,39,23.0,6.0,3.0,NaN,...,NaN,0.925,4.0,53.0,41,38,8,3,NaN,NaN
3,Kyle Datres,2017,3,0.391,32.0,48,51.0,4.0,2.0,NaN,...,NaN,0.959,6.0,63.0,63,61,20,7,NaN,NaN
4,Brandon Martorano,2017,4,0.208,91.0,5,5.0,0.0,1.0,NaN,...,NaN,1.000,3.0,36.0,14,3,2,0,NaN,NaN
5,Ashton McGee,2017,5,0.417,21.0,45,46.0,1.0,5.0,NaN,...,NaN,0.943,4.0,58.0,57,73,2,7,NaN,NaN
6,Ike Freeman,2017,8,0.286,24.0,13,8.0,0.0,2.0,NaN,...,NaN,0.974,0.0,39.0,14,8,1,1,NaN,NaN
7,Zack Gahagan,2015,10,0.291,187.0,19,32.0,3.0,2.0,NaN,...,NaN,0.969,3.0,52.0,41,33,3,2,NaN,NaN
8,Zack Gahagan,2016,10,0.391,151.0,24,34.0,1.0,2.0,NaN,...,NaN,0.946,3.0,51.0,42,44,2,5,NaN,NaN
9,Zack Gahagan,2017,10,0.366,237.0,45,42.0,6.0,4.0,NaN,...,NaN,0.974,4.0,62.0,61,51,10,7,NaN,NaN
